### Connection of the ASTRA DB

Please Ensure to enter your own Token and ID

In [ ]:
import json
import cassio
with open("./keys.json", "r") as file:
    data = json.load(file)
ASTRA_DB_APPLICATION_TOKEN=data['ASTRA_DB_APPLICATION_TOKEN']
ASTRA_DB_ID=data['ASTRA_DB_ID']
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

Loading the pdf document and Splitting the text

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("input_rag_merged.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=100)
documents=text_splitter.split_documents(docs)

Generating Text Embeddings and Saving the Astra DB

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.cassandra import Cassandra

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
astra_vector_store=Cassandra(embedding=embeddings,table_name="qa_mini_demo",session=None,keyspace=None)

from langchain.indexes.vectorstore import VectorStoreIndexWrapper
astra_vector_store.add_documents(documents)
print("Inserted %i headlines." % len(documents))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)